In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
df3 = pd.read_csv("/content/drive/MyDrive/ESAA_OB/project1/df3.csv")

<ipython-input-3-e8675424591f>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("/content/drive/MyDrive/ESAA_OB/project1/df3.csv")


#모델링 전 데이터 작업

In [ ]:
#train, test 데이터 나누기
train = df3[:26457]
test = df3[26457:].reset_index(drop=True)

#test 데이터에서 credit 변수 드랍하기
test.drop(labels=["credit"],axis=1, inplace=True)

In [ ]:
train['credit'].unique()

array([1.0, 2.0, 0.0, '2.0', '0.0', '1.0'], dtype=object)

In [ ]:
train["credit"] = train["credit"].astype(float).astype(int)

<ipython-input-6-cd75d914b685>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["credit"] = train["credit"].astype(float).astype(int)


In [ ]:
train['credit'].unique()

array([1, 2, 0])

In [ ]:
train['credit'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26457 entries, 0 to 26456
Series name: credit
Non-Null Count  Dtype
--------------  -----
26457 non-null  int64
dtypes: int64(1)
memory usage: 206.8 KB


In [ ]:
Y_train = train["credit"]
X_train = train.drop(labels = ["credit"],axis=1)

# 개별 모델링 - (stratified)kfold=10

## 하이퍼파라미터x

In [ ]:
#from sklearn.model_selection import train_test_split

#X_train, x_val, y_train, y_train_val = train_test_split(X_train,Y_train, test_size=0.2, random_state=10)

In [ ]:
# 임포트
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,  AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

In [ ]:
# kfold 설정
kfold = StratifiedKFold(n_splits=10)

In [ ]:
rs = 10
classifiers = []
classifiers.append(RandomForestClassifier(random_state=rs))
classifiers.append(ExtraTreesClassifier(random_state=rs))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=rs),random_state=rs,learning_rate=0.1))

In [ ]:
classifiers

[RandomForestClassifier(random_state=10),
 ExtraTreesClassifier(random_state=10),
 AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=10),
                    learning_rate=0.1, random_state=10)]

In [ ]:
Algorithm_Names = ["RandomForest","ExtraTrees","AdaBoost"]

In [ ]:
cv_results = []
for classifier in classifiers:
  cv_results.append(cross_val_score(classifier, X_train, y=Y_train, scoring="accuracy",cv=kfold, n_jobs=4))

cv_means=[]
cv_std = []
for cv_result in cv_results:
  cv_means.append(cv_result.mean())
  cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"Algorithm": Algorithm_Names, "CrossValMeans":cv_means,"CrossValerros":cv_std})

In [ ]:
cv_res

,Algorithm,CrossValMeans,CrossValerros
0,RandomForest,0.700835,0.006110
1,ExtraTrees,0.681785,0.004668
2,AdaBoost,0.668896,0.006059


## 하이퍼파라미터 튜닝   

### RandomForest

In [ ]:
#RandomForestClassifier
RFC = RandomForestClassifier()

rf_param_grid = {"max_depth": [None],
              "min_samples_split": [3, 5, 10],
              "min_samples_leaf": [1, 3, 5],
              "n_estimators" :[100,300,400],
              "criterion": ["log_loss"]}

gsRFC = GridSearchCV(RFC, param_grid=rf_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose=1)

gsRFC.fit(X_train, Y_train)

RFC_best = gsRFC.best_estimator_

#Best score
gsRFC.best_score_

Fitting 10 folds for each of 27 candidates, totalling 270 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0.7261215488085594

In [ ]:
#Best parameter
gsRFC.best_params_

{'criterion': 'log_loss',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 300}

In [ ]:
RFC_best.fit(X_train, Y_train)
pred_gsRFC = RFC_best.predict_proba(test)

In [ ]:
pred_gsRFC

array([[0.05862289, 0.1889468 , 0.75243031],
       [0.23613532, 0.28476564, 0.47909904],
       [0.0593029 , 0.09801987, 0.84267723],
       ...,
       [0.01741911, 0.1161841 , 0.86639679],
       [0.21206253, 0.36113728, 0.42680019],
       [0.11132886, 0.25630531, 0.63236583]])

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/ESAA_OB/project1/sample_submission.csv')
submission

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0
...,...,...,...,...
9995,36452,0,0,0
9996,36453,0,0,0
9997,36454,0,0,0
9998,36455,0,0,0


In [ ]:
submission.iloc[:,1:]=pred_gsRFC
submission

,index,0,1,2
0,26457,0.058623,0.188947,0.752430
1,26458,0.236135,0.284766,0.479099
2,26459,0.059303,0.098020,0.842677
3,26460,0.093918,0.088154,0.817928
4,26461,0.092073,0.189506,0.718421
...,...,...,...,...
9995,36452,0.091635,0.302908,0.605457
9996,36453,0.251407,0.301551,0.447042
9997,36454,0.017419,0.116184,0.866397
9998,36455,0.212063,0.361137,0.426800


In [ ]:
# csv 파일로 저장
submission.to_csv('/content/gsRFC.csv', index=False)

데이콘 점수
- public: 0.699278573
- private: 0.6812564501

### ExtraTrees

In [ ]:
#ExtraTrees
ExtC = ExtraTreesClassifier()

## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "min_samples_split": [3, 5, 10],
              "min_samples_leaf": [1, 3, 5],
              "n_estimators" :[100,300,400],
              "criterion": ["log_loss"]}

gsExtC = GridSearchCV(ExtC, param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose=1)

gsExtC.fit(X_train, Y_train)

ExtC_best = gsExtC.best_estimator_

#Best score
gsExtC.best_score_

Fitting 10 folds for each of 27 candidates, totalling 270 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0.7141775080122366

In [ ]:
#Best parameter
gsExtC.best_params_

{'criterion': 'log_loss',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 300}

In [ ]:
RFC_best.fit(X_train, Y_train)
pred_gsExtC = RFC_best.predict_proba(test)

In [ ]:
pred_gsExtC

array([[0.06332475, 0.19108799, 0.74558726],
       [0.24440454, 0.27619652, 0.47939894],
       [0.04954852, 0.07388468, 0.87656681],
       ...,
       [0.02317381, 0.13838425, 0.83844194],
       [0.20736448, 0.38534452, 0.40729101],
       [0.1437413 , 0.23801478, 0.61824392]])

In [ ]:
submission.iloc[:,1:]=pred_gsExtC
submission

,index,0,1,2
0,26457,0.063325,0.191088,0.745587
1,26458,0.244405,0.276197,0.479399
2,26459,0.049549,0.073885,0.876567
3,26460,0.098457,0.094347,0.807196
4,26461,0.096546,0.203665,0.699790
...,...,...,...,...
9995,36452,0.093197,0.256196,0.650607
9996,36453,0.272900,0.236862,0.490239
9997,36454,0.023174,0.138384,0.838442
9998,36455,0.207364,0.385345,0.407291


In [ ]:
# csv 파일로 저장
submission.to_csv('/content/gsExtC.csv', index=False)

데이콘 점수
- public: 0.6992200846
- private: 0.6796305667